# Prediction Pipeline

In [120]:
import pandas as pd
import numpy as np
import joblib
from IPython.display import display
import re
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    text = re.sub(r'[^\w\s]', ' ', text.lower())
    words = text.split()
    return ' '.join(lemmatizer.lemmatize(w) for w in words if w.isalpha())

X_predict_raw_arr = [
    "question access check message support report info review message report",
    "policy problem notice error message urgent support",
    "",
    "thanks question access support update info question",
    "Hi, just wanted to notice that there’s an urgent problem. I tried to check the status, but the update seems delayed. Please fix this and let me know, I already sent a report earlier.",
    ""
]

X_predict_raw = pd.DataFrame(X_predict_raw_arr, columns=["text"])

X_predict_raw.replace("", float("NaN"), inplace=True)
X_predict_raw = X_predict_raw.dropna(how='any').reset_index(drop=True)

X_predict = []

for index, row in X_predict_raw.iterrows():
  X_predict.append(lemmatize_text(row['text']))

tfidf = joblib.load("tfidf.pkl")

X_transformed = tfidf.transform(X_predict)
model = joblib.load("model.pkl")

y_pred = model.predict(X_transformed)
conf = model.predict_proba(X_transformed)

y_conf = []
for i in range(len(conf[0])):
  y_conf.append([conf[0][i][0], conf[1][i][0], conf[2][i][0], conf[3][i][0]])

df = pd.DataFrame(X_predict_raw, columns=["text"])
df_pred = pd.DataFrame(y_pred, columns=["label_1", "label_2", "label_3", "label_4"])
df_conf = pd.DataFrame(y_conf, columns=["conf_1", "conf_2", "conf_3", "conf_4"])

df = pd.concat([df, df_pred, df_conf], axis=1)

display(df)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,label_1,label_2,label_3,label_4,conf_1,conf_2,conf_3,conf_4
0,question access check message support report i...,0,0,0,1,0.714970,0.539458,0.560667,0.352609
1,policy problem notice error message urgent sup...,1,1,0,1,0.289869,0.175650,0.715861,0.398913
2,thanks question access support update info que...,0,0,0,0,0.667931,0.685171,0.684203,0.695767
3,"Hi, just wanted to notice that there’s an urge...",1,1,1,1,0.282467,0.105284,0.250238,0.396758
